In [132]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Visualization

In [133]:
import pandas as pd
import seaborn as sns

In [134]:
df = pd.read_csv('../train.csv')

In [135]:
df.shape

(250000, 32)

In [136]:
df[df.Prediction=='s'].shape

(85667, 32)

In [137]:
df[df.Prediction=='b'].shape

(164333, 32)

In [115]:
feat_names = [col for col in df.columns if col not in ['Id','Prediction']]

#for colname in feat_names:
#    df.loc[df[colname]==-999.000,[colname]]=df[colname][df[colname]!=-999.000].median()

In [116]:
df.describe()

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,224999.500000,120.417434,49.239819,81.181982,57.895962,2.193104,268.220619,-0.411629,2.373100,18.917332,...,-0.010119,209.797178,0.979176,77.124366,-0.001966,-0.020629,50.739149,-0.010535,-0.001879,73.064591
std,72168.927986,52.859218,35.344886,40.828691,63.655682,0.948104,224.229668,1.948520,0.782911,22.273494,...,1.812223,126.499506,0.977426,47.939899,1.382703,1.405084,17.792135,1.094446,0.978743,98.015662
min,100000.000000,9.044000,0.000000,6.329000,0.000000,0.000000,13.602000,-18.066000,0.208000,0.000000,...,-3.142000,13.678000,0.000000,30.000000,-4.499000,-3.142000,30.000000,-4.500000,-3.142000,0.000000
25%,162499.750000,95.665000,19.241000,59.388750,14.068750,2.107000,225.885000,-0.244000,1.810000,2.841000,...,-1.575000,123.017500,0.000000,57.439000,-0.433000,-0.556000,47.902000,-0.010000,-0.002000,0.000000
50%,224999.500000,112.406000,46.524000,73.752000,38.467500,2.107000,225.885000,-0.244000,2.491500,12.315500,...,-0.024000,179.739000,1.000000,65.561000,0.000000,-0.033000,47.902000,-0.010000,-0.002000,40.512500
75%,287499.250000,130.606250,73.598000,92.259000,79.169000,2.107000,225.885000,-0.244000,2.961000,27.591000,...,1.561000,263.379250,2.000000,75.349000,0.433000,0.503000,47.902000,-0.010000,-0.002000,109.933750
max,349999.000000,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,...,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


In [138]:
df.shape

(250000, 32)

In [139]:
sum(df["DER_mass_MMC"]==-999.00)

38114

In [45]:
df.to_csv('../train_mod.csv', index=False)

In [58]:
DER_features = [colname for colname in df.columns if colname.startswith("DER")]

In [59]:
phys_feats = ['Id','Prediction'] + DER_features

In [60]:
df_phys = df[phys_feats]

In [62]:
df_phys.to_csv('../train_phys.csv', index=False)

In [66]:
df[DER_features].as_matrix().shape

(250000, 13)

In [78]:
test = pd.read_csv('../test.csv')
test_phys = test[phys_feats]

In [82]:
test_phys.to_csv('../test_phys.csv',index=False)

## PCA

In [140]:
M = df[feat_names].as_matrix()
U, s, Vt = np.linalg.svd(M, full_matrices=False)
V = Vt.T

# PCs are already sorted by descending order 
# of the singular values (i.e. by the
# proportion of total variance they explain)

# if we use all of the PCs we can reconstruct the noisy signal perfectly
S = np.diag(s)
Mhat = np.dot(U, np.dot(S, V.T))
print("Using all PCs, MSE = %.6G" %(np.mean((M - Mhat)**2)))

# if we use only the first 20 PCs the reconstruction is less accurate
Mhat2 = np.dot(U[:, :20], np.dot(S[:20, :20], V[:,:20].T))
print("Using first 20 PCs, MSE = %.6G" %(np.mean((M - Mhat2)**2)))

Using all PCs, MSE = 3.88932E-24
Using first 20 PCs, MSE = 0.177652


In [141]:
singulars = np.diag(S)**2
singulars.cumsum()/singulars.sum()

array([ 0.87446117,  0.94964099,  0.97401173,  0.99471526,  0.99870389,
        0.99912491,  0.99940475,  0.99959834,  0.9997434 ,  0.9998227 ,
        0.99988145,  0.99993866,  0.99996817,  0.99998939,  0.999997  ,
        0.99999761,  0.99999814,  0.99999856,  0.99999894,  0.99999919,
        0.9999994 ,  0.99999956,  0.99999968,  0.99999979,  0.99999989,
        0.99999996,  0.99999998,  0.99999999,  1.        ,  1.        ])

In [142]:
up_to = 5
Mhat = np.dot(U[:, :up_to], np.dot(S[:up_to, :up_to], V[:,:up_to].T))
Mhat.shape

(250000, 30)

In [143]:
df_pca = pd.DataFrame(Mhat)
df_pca['Id'] = df['Id']
df_pca['Prediction'] = df['Prediction']
df_pca.columns = feat_names + ['Id','Prediction']
df_pca = df_pca[['Id','Prediction']+feat_names]
df_pca.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.874048,19.066899,48.594329,83.251886,1.886396,124.695484,1.342045,1.008665,...,0.007373,239.023662,1.307359,75.093247,-4.509994,-4.511033,38.002589,1.025672,1.024730,133.529823
1,100001,b,159.698839,36.931498,78.172665,55.297834,-998.534687,-998.490331,-998.665065,2.309680,...,-0.002444,171.750439,0.811787,51.945018,-2.556009,-2.571503,-1003.706321,-998.490310,-998.490224,46.381219
2,100002,b,-999.648397,88.892883,73.266044,79.495140,-1001.654061,-999.166789,-1001.400750,1.854753,...,-0.032879,237.816882,1.267982,67.604991,-14.446781,-14.453713,-993.513282,-1001.697945,-1001.692815,94.921891
3,100003,b,141.801663,45.167866,83.993699,7.574865,-998.580553,-998.251360,-998.693283,2.805168,...,-0.015908,110.196134,-0.108426,-1003.890815,-996.522083,-996.524288,-1001.831558,-998.567339,-998.568892,-12.444934
4,100004,b,177.558781,41.809811,80.900363,0.980680,-998.151920,-997.724931,-998.339127,2.758815,...,-0.015205,91.226817,-0.210658,-1007.865859,-994.129380,-994.131863,-1004.300521,-998.113294,-998.114781,-23.309099


In [144]:
df_pca.shape

(250000, 32)

In [145]:
df_pca.to_csv('../train_pca.csv',index=False)

## PCA test

In [146]:
df_pca_test = pd.read_csv('../test.csv')

In [147]:
M = df_pca_test[feat_names].as_matrix()
U, s, Vt = np.linalg.svd(M, full_matrices=False)
V = Vt.T
S = np.diag(s)

In [148]:
singulars = np.diag(S)**2
singulars.cumsum()/singulars.sum()

array([ 0.8744866 ,  0.94948899,  0.97392872,  0.99468636,  0.99871676,
        0.99913602,  0.99941321,  0.99960441,  0.99974825,  0.99982542,
        0.99988318,  0.99993897,  0.99996824,  0.99998935,  0.99999701,
        0.99999761,  0.99999815,  0.99999856,  0.99999894,  0.99999919,
        0.9999994 ,  0.99999955,  0.99999968,  0.99999979,  0.99999989,
        0.99999996,  0.99999998,  0.99999999,  1.        ,  1.        ])

In [149]:
up_to = 5
Mhat = np.dot(U[:, :up_to], np.dot(S[:up_to, :up_to], V[:,:up_to].T))
Mhat.shape

(568238, 30)

In [150]:
pca_test = pd.DataFrame(Mhat)
pca_test['Id'] = df_pca_test['Id']
pca_test['Prediction'] = df_pca_test['Prediction']
pca_test.columns = feat_names + ['Id','Prediction']
pca_test = pca_test[['Id','Prediction']+feat_names]
pca_test.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,350000,?,-999.313507,87.826337,62.996786,3.445537,-998.206178,-999.255479,-998.185191,2.053598,...,-0.046573,94.129632,-0.083393,-1003.042161,-996.344262,-996.340544,-1000.327116,-998.155610,-998.151046,-9.596692
1,350001,?,105.878379,39.827035,78.463781,57.432250,-998.534304,-998.745688,-998.635804,2.298334,...,-0.000693,177.207397,0.844583,52.816502,-3.826337,-3.847365,-1002.779514,-998.495823,-998.495018,49.971350
2,350002,?,117.604228,48.151296,87.181584,14.473287,-999.144604,-998.252831,-999.191254,2.863133,...,-0.018005,129.857268,-0.003315,-999.650326,-998.830319,-998.831189,-999.436746,-999.159323,-999.160941,-1.310420
3,350003,?,136.924130,49.648821,91.642041,21.979736,-999.528075,-999.136922,-999.493119,2.952134,...,-0.017242,151.755920,0.108462,-995.067268,-1001.275388,-1001.277091,-996.650166,-999.570601,-999.573014,10.626741
4,350004,?,74.810040,81.782869,163.295174,289.396701,14.347399,537.136044,11.468196,3.322684,...,0.021757,833.457127,4.641016,271.044506,-9.567698,-9.602550,143.603728,10.967215,10.947979,472.607369


In [151]:
pca_test.to_csv('../test_pca.csv',index=False)